<a href="https://colab.research.google.com/github/JacobDowns/CSCI-491-591/blob/main/lecture6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Advanced mpi4py Features

* In this lecture we'll cover a handful of advanced features in mpi4py
* We'll discuss parallel I/O, persistent communication, and one sided communication

### Persistent Communication
* In some of examples, particularly the heat equation, we sent many of the same type of message repreatedly in a loop
* In such cases, communication can be optimized by using persistent communication, a particular case of nonblocking communication allowing the reduction of the overhead
* For point-to-point communication, persistent communication is used by setting up requests with `Send_init` and Recv_init`
* In each loop iteration, you would then call `Start` or `Startall` and subsequently `Wait` or `Waitall`



#### 1. Usage Pattern
* Create a request one time
```python
req_s = comm.Send_init(buf, dest=..., tag=...)
req_r = comm.Recv_init(buf, source=..., tage=...)
```
* In a loop you can repeat the message with the outlines form many times
```python
req_s.Start()
req_r.Start()
MPI.Request.Waitall([req_s, req_r])
```
After you're finished sending messages, clean up with
```python
req_s.Free()
req_r.Free()
```
* It's a little funky to have to free something in a Python program, but a persistent request creates a `request` that holds onto
  * A pointer to the buffer
  * Datatype description
  * The communicator and tag
* Free will tell MPI you're done with these resources and is another reminder of how MPI is a lower level library being wrapped in Python


#### 2. Example: Sending Data in a Ring!
* Below, let's look at an example where we have each rank send some information to its left, wrapping around to the last rank

In [2]:
%%bash
cat > hello_mpi.py <<'PY'
from mpi4py import MPI
import numpy as np

comm  = MPI.COMM_WORLD
rank  = comm.Get_rank()
size  = comm.Get_size()

right = (rank + 1) % size
left  = (rank - 1) % size

sendbuf = np.array(rank, dtype='i')   # Will send our rank
recvbuf = np.array(-1, dtype='i')     # Will receive from 'left'

# Build persistent requests once
send_req = comm.Send_init(sendbuf, dest=right, tag=0)
recv_req = comm.Recv_init(recvbuf,  source=left,  tag=0)

n_iters = 5
for it in range(n_iters):
    # Optionally update what we send each iter
    sendbuf[...] = rank + 100*it

    # Start both; then wait for both
    send_req.Start()
    recv_req.Start()
    MPI.Request.Waitall([send_req, recv_req])

    print(f"[iter {it}] rank {rank} got {recvbuf} from {left}")

send_req.Free()
recv_req.Free()
PY

In [3]:
!mpiexec -n 4 python hello_mpi.py

[iter 0] rank 0 got 3 from 3
[iter 1] rank 0 got 103 from 3
[iter 2] rank 0 got 203 from 3
[iter 3] rank 0 got 303 from 3
[iter 4] rank 0 got 403 from 3
[iter 0] rank 1 got 0 from 0
[iter 1] rank 1 got 100 from 0
[iter 2] rank 1 got 200 from 0
[iter 3] rank 1 got 300 from 0
[iter 4] rank 1 got 400 from 0
[iter 0] rank 2 got 1 from 1
[iter 1] rank 2 got 101 from 1
[iter 2] rank 2 got 201 from 1
[iter 3] rank 2 got 301 from 1
[iter 4] rank 2 got 401 from 1
[iter 0] rank 3 got 2 from 2
[iter 1] rank 3 got 102 from 2
[iter 2] rank 3 got 202 from 2
[iter 3] rank 3 got 302 from 2
[iter 4] rank 3 got 402 from 2


* What fun, we sent some data in a ring!
* There is potentially a slight performance benefit from persistent communication


## One-Sided Communication
* We have stated that MPI doesn't use a shared memory paradigm, hence the necessity of sending messages
* However, this is not entirely true, as MPI supports a on-sided communication model using **Remote Memmory Access** (RMA)
    * In RMA one cprocess (the origin) directly reads from or writes into memory exposed by another procces (the target)
    * The target process doesn't need to call a send / receive on the other end
* In mpi4py, one-sided operations are available using windows via the `Win` 
* The main operations for windows are:
    * `Put`: write data into a target's windows
    * `Get` : Read data from a target's window
    * `Accumulate` : atomic fetch and combine into target (sum, max, etc.)


#### 1. Synchronization
* You'll probably notice that this paradigm is much like a shared memory model for a multithreaded application
* This comes with many of the same challenges as threading including race conditions
* RMA has a couple primary synchronization mechanisms:
    * **Fence**: collective barrier-like synchronization
    * **Lock / Unlock**: finer control for accessing one target at a time
* RMA is not a true shared memory model, but it behaves much like one

#### 2. Examples

* This first example is very basic 
* Each process exposes a single integer which can be read or written to
* Rank 0 writes into rank 1's window
* `Fence` is used for synchronization much like `comm.Barrier()` 

In [5]:
%%bash
cat > hello_mpi.py <<'PY'
from mpi4py import MPI
import numpy as np

comm = MPI.COMM_WORLD
rank = comm.Get_rank()

# Each process exposes one integer
buf = np.array(rank, dtype='i')
win = MPI.Win.Create(buf, comm=comm)

if rank == 0:
    data = np.array(42, dtype='i')
    win.Fence()
    win.Put([data, MPI.INT], target=1)   # rank 0 writes into rank 1
    win.Fence()
elif rank == 1:
    win.Fence()
    win.Fence()  # wait for data arrival
    print(f"Rank 1 sees {buf[0]}")

win.Free()
PY